In [1]:
import sys
import os
import numpy as np
from pathlib import Path
os.chdir(Path(os.getcwd()).resolve().parents[1])
import setup
from methods import grid_selection_amanda_dynamic
from sklearn.model_selection import ParameterGrid



def writeResults(datasetID, bestScore, bestParams, clfName, distanceMetric, beta=None):
    if distanceMetric == 'BBD' and isinstance(beta,list):
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}-ALLBETAS.txt".format(distanceMetric,clfName)
    elif distanceMetric == 'BBD':
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}{:.3f}_EVL-{}.txt".format(distanceMetric, beta,clfName)
    else:
        path = "results/batch/dynamic/gridsearch_amanda_dynamic_{}_EVL-{}.txt".format(distanceMetric,clfName)
        
    file = open(path,"a") 
    string = "{}: {} using {} \n".format(datasetID, bestScore, bestParams)
    file.write(string)
    file.close() 


def main(distanceMetric, beta=None):
    is_windows = sys.platform.startswith('win')
    sep = '\\'
    
    if is_windows == False:
        sep = '/'

    path = os.getcwd()+sep+'data'+sep
    
    #loading datasets
    datasets = [setup.loadCDT, setup.loadCHT, setup.load2CDT, setup.load2CHT, setup.load4CR, setup.load4CRE_V1, 
                setup.load4CRE_V2, setup.load5CVT, setup.loadCSurr, setup.load4CE1CF, setup.loadUG_2C_2D, setup.loadMG_2C_2D, 
                setup.loadFG_2C_2D, setup.loadUG_2C_3D, setup.loadUG_2C_5D, setup.loadGEARS_2C_2D, setup.loadCheckerBoard, 
                setup.loadElecData, setup.loadNOAADataset, setup.loadKeystroke]
    
    #arrClfName = ['SGD', 'NB', 'RF', 'LP', 'KNN']
    arrClfName = ['LP']
    if (distanceMetric == 'BBD') and (beta is None):
        raise Exception(("AMANDA-DCP with BBD must have a beta value. beta = {}").format(str(beta)))
    
    if beta is list:
        allBetas = True
    else:
        allBetas=False
        
    for clfName in arrClfName:
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("**************** BEGIN of {}-{}{:.3f} results ****************".format(clfName, distanceMetric, beta))            
        else:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))

        poolSize = None
        isBatchMode = True
        #testing grid search
        for i in range(len(datasets)):
            batches=47
            if i==len(datasets)-2:
                batches=24
            elif i==len(datasets)-1:
                batches=4
            
            finalScore = 0
            best_grid={}
            dataValues, dataLabels, description = datasets[i](path, sep)

            #Train-test split
            availableQty = int(0.5*len(dataLabels))
            availableLabels = dataLabels[:availableQty] 
            availableData = dataValues[:availableQty]

            # 70/30 train/test data
            initialLabeledData = int(0.1*len(availableLabels))
            sizeOfBatch = int((len(availableLabels)-initialLabeledData)/batches)

            print("{}: {} batches of {} instances".format(description, batches, sizeOfBatch))

            tuned_params = [{"sizeOfBatch":[sizeOfBatch], 
                             "batches":[batches], "poolSize":[poolSize], "isBatchMode":[isBatchMode], 
                             "initialLabeledData":[initialLabeledData], "clfName":[clfName],
                            "distanceMetric":[distanceMetric], "beta":[beta]}]
            if clfName == 'LP' or clfName == 'KNN':
                tuned_params[0].update({"K":[2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]})

            for g in ParameterGrid(tuned_params):
                averageAccuracy=0
                gs = grid_selection_amanda_dynamic.run(**g)

                try:
                    gs.fit(availableData, availableLabels)
                    averageAccuracy = np.mean(gs.predict())
                    print(averageAccuracy, g)
                    if finalScore < averageAccuracy:
                        finalScore = averageAccuracy
                        best_grid = g
                except Exception:
                    print("An error occured in ", description, g)
                    #raise Exception

            print(finalScore)
            print(best_grid)
            print("=======================================================================================================")

            writeResults(description, finalScore, best_grid, clfName, distanceMetric, beta)
        
        if distanceMetric == 'BBD' and allBetas:
            print("**************** BEGIN of {}-{} results ****************".format(clfName, distanceMetric))
        elif distanceMetric == 'BBD':
            print("******** END of {}-{}{:.3f} results ********".format(clfName, distanceMetric, beta))
        else:
            print("******** END of {}-{} results ********".format(clfName, distanceMetric))
    
if __name__ == "__main__":
    distanceMetric = 'BBD'
    betas = [-50, -1, -0.5, -0.1, 1.001, 1.1, 2, 3, 5, 10, 50]
    
    # Grid search for each beta value - BBD
    for beta in betas:
        main(distanceMetric, beta)
        print('\n\n\n')
        
    # grid search for Hellinger
    distanceMetric = 'Hellinger'
    main(distanceMetric, beta)
    print('\n\n\n')
    
    distanceMetric = 'Hellinger2'
    main(distanceMetric, beta)
    print('\n\n\n')


**************** BEGIN of LP-BBD-50.000 results ****************
One Class Diagonal Translation. 2 Dimensional data.: 47 batches of 153 instances
99.36127659574468 {'K': 2, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.36127659574468 {'K': 3, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.34723404255321 {'K': 4, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.34723404255321 {'K': 5, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
99.37510638297874 {'K': 6, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800

52.565531914893626 {'K': 7, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
52.648510638297886 {'K': 8, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
52.50978723404256 {'K': 9, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
53.107659574468094 {'K': 10, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
53.19106382978725 {'K': 11, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
53.06574468085107 {'K': 12, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distan

23.2731914893617 {'K': 2, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
25.522553191489354 {'K': 3, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
22.883191489361703 {'K': 4, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
23.506382978723398 {'K': 5, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
23.54361702127659 {'K': 6, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
23.441276595744675 {'K': 7, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'dist

50.1655319148936 {'K': 8, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
41.393191489361705 {'K': 9, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
41.214893617021275 {'K': 10, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
41.24872340425532 {'K': 11, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
39.88787234042554 {'K': 12, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
50.1655319148936
{'K': 8, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'dist

Two 5-dimensional Unimodal Gaussian Classes.: 47 batches of 1914 instances
71.86702127659575 {'K': 2, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
63.431702127659584 {'K': 3, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
74.65042553191489 {'K': 4, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
63.48914893617022 {'K': 5, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
63.66170212765958 {'K': 6, 'batches': 47, 'beta': -50, 'clfName': 'LP', 'distanceMetric': 'BBD', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 

...

52.32872340425533 {'K': 12, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
52.60702127659576
{'K': 10, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 800, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 153}
Four Classes Rotating Separated. Bidimensional.: 47 batches of 1382 instances
28.390851063829786 {'K': 2, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1382}
28.435744680851066 {'K': 3, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 7220, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1382}
28.167872340425536 {'K': 4, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 7220, 'isBatchMode': True, 'pool

20.9968085106383 {'K': 3, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
21.407021276595746 {'K': 4, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
21.6568085106383 {'K': 5, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
21.64723404255319 {'K': 6, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
21.638297872340427 {'K': 7, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1200, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 229}
21.63787234042553 {'K': 8, 'batches': 47, 'beta': 

43.33574468085106 {'K': 8, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
42.58893617021276 {'K': 9, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
43.367446808510635 {'K': 10, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
41.23489361702127 {'K': 11, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
42.15510638297872 {'K': 12, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 5000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 957}
52.46872340425531
{'K': 5, 'batches': 47, 'bet

69.59446808510639 {'K': 12, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
71.36255319148937
{'K': 2, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
Two 5-dimensional Unimodal Gaussian Classes.: 47 batches of 1914 instances
58.78446808510639 {'K': 2, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
70.00702127659574 {'K': 3, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 1914}
68.10489361702128 {'K': 4, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 10000, 'isBatchMode': True, 'po

44.46191489361702 {'K': 3, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 263}
64.97085106382978 {'K': 4, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 263}
59.93787234042553 {'K': 5, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 263}
64.84106382978723 {'K': 6, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 263}
62.69702127659574 {'K': 7, 'batches': 47, 'beta': 50, 'clfName': 'LP', 'distanceMetric': 'Hellinger2', 'initialLabeledData': 1377, 'isBatchMode': True, 'poolSize': None, 'sizeOfBatch': 263}
65.90914893617023 {'K': 8, 'batches': 47, 'beta': 

# 